In [20]:
import torch
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [21]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-27 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [22]:
def detect_and_save_objects(image_path, save_path, target_class=None):
    """
    Detect objects in an image, overlay them on a white background with bounding boxes, and save the result.

    Parameters:
    - image_path: Path to the input image.
    - save_path: Path to save the output image.
    - target_class: Class ID of the objects to detect and overlay. If None, all detected objects are used.
    """
    # Load image
    img = Image.open(image_path)

    # Perform object detection
    results = model(img)
    detections = results.pred[0].cpu().numpy()

    # Filter detections by target_class if provided
    if target_class is not None:
        detections = [det for det in detections if det[5] == target_class]

    print(f'Number of objects detected: {len(detections)}')

    # Create a white background image
    white_bg = np.ones_like(np.array(img)) * 255

    # Overlay each detected object with its bounding box on the white background
    for *box, conf, cls in detections:
        x1, y1, x2, y2 = map(int, box)
        object_img = np.array(img)[y1:y2, x1:x2]
        white_bg[y1:y2, x1:x2] = object_img

        # Draw bounding box
        cv2.rectangle(white_bg, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Convert the white background image to PIL Image and display
    white_bg_image = Image.fromarray(white_bg.astype('uint8'))
    plt.imshow(white_bg_image)
    plt.axis('off')  # Hide axes
    plt.show()

    # Save the image
    white_bg_image.save(save_path)
    print(f"Image saved successfully at {save_path}")

In [24]:
detect_and_save_objects('/content/cars.jpg', '/content/cars_with_bounding_boxes.jpg', target_class=2)  # Detect cars
detect_and_save_objects('/content/pedestrians.jpg', '/content/pedestrians_with_bounding_boxes.jpg')    # Detect all objects

Number of objects detected: 6
Image saved successfully at /content/cars_with_bounding_boxes.jpg
Number of objects detected: 20
Image saved successfully at /content/pedestrians_with_bounding_boxes.jpg
